In [1]:
#!pip install onnx

In [2]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

tensor([1, 2, 3], device='cuda:0')

In [3]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,
                            DataLoader,
                            TensorDataset)
import tqdm

In [5]:
from torchvision import models

def create_network():
    
    # resnet18 기반의 이종 분류 식별 모델
    net = models.resnet18()
    fc_input_dim = net.fc.in_features
    net.fc = nn.Linear(fc_input_dim, 2)
    return net

# 모델 생성
net = create_network()

# 파라미터 읽기 및 모델에 설정
prm = torch.load("WebAPI/taco_burrito.prm", map_location="cpu")
net.load_state_dict(prm)

# 평가 모드로 설정
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [6]:
import torch.onnx
dummy_data = torch.empty(1, 3, 224, 224, dtype=torch.float32)
torch.onnx.export(net, dummy_data, "taco_burrito.onnx")

In [9]:
import onnx
from caffe2.python.onnx import backend as caffe2_backend

# ONNX 모델 불러오기
onnx_model = onnx.load("taco_burrito.onnx")

# ONNX 모델을 Caffe2 모델로 변환
backend = caffe2_backend.prepare(onnx_model)

In [10]:
from PIL import Image
from torchvision import transforms

# 이미지를 잘라서 Tensor로 변환하는 함수
transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# 이미지 불러오기
img = Image.open("WebAPI/360.jpg")

# Tensor로 변환해서 배치 차원을 더함
img_tensor = transform(img).unsqueeze(0)

# ndarray로 변환
img_ndarray = img_tensor.numpy()

# 파이토치로 실행
net(img_tensor)

tensor([[ 1.5830, -1.3071]], grad_fn=<AddmmBackward>)

In [11]:
# ONNX/Caffe2로 실행
output = backend.run(img_ndarray)
output[0]

array([[ 1.5829796, -1.3070935]], dtype=float32)

In [12]:
from caffe2.python.onnx.backend import Caffe2Backend

init_net, predict_net = \
    Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model)

In [13]:
with open('init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())

with open('predict_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())